In [7]:
HTML(read(open("style.html"), String))

HTML{String}("<link href='https://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='https://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('https://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\", sans-serif;\r\n    font-size: 100%;\r\n}\r\n\r\n</style>")

In [8]:
import Chess
using NBInclude

In [9]:
@nbinclude "4.0 - Memoisierung - Zobrist-Hashing.ipynb"

toggleColor! (generic function with 1 method)

# Extended Board

Dieses Notebook beschreibt die Struktur `ExtendedBoard`. Sie erzeugt aus einem normalen Schachbrett, dem `Chess.Board` ein Spielbrett, welches weitere Parameter mitführt. Die Notwendigkeit des `ExtendedBoard` begründet sich underanderem mit zusätzlichen Regeln, welche überprüft werden müssen. 

Im Schach gibt es die `Threefold repetition rule`, die Regel der dreifachen Wiederholung. Diese besagt, dass ein Spieler das Recht auf ein Unentschieden hat, sobald sich eine Stellung dreimal im Spiel wiederholt. Dabei gilt, dass alle Figuren auf den gleichen Spielfeldern stehen. Die Wiederholungen müssen nicht direkt nacheinander geschehen. 

Die Überprüfung dieser Regel ist nicht in der `Chess.jl` Bibliothek enthalten. Der [Zobrist-Hash](4.0%20-%20Memoisierung%20-%20Zobrist-Hashing.ipynb) wird genutzt, um Häufigkeiten aller Stellungen unter entsprechenden Schlüsseln zu speichern und Wiederholungen festzuhalten. Daher werden folgende Attribute zusätzlich von uns implementiert:

+ score &rarr; die Bewertung des Spiels
+ endGame &rarr; die Überpüfung der Endspiel-Kriterien
+ repetions &rarr; das Dictionary der wiederholten Stellungen
+ repetionRuleDraw &rarr; die Überprüfung der Regel der dreifachen Wiederholung
+ zobrist &rarr; der Hash des Spielbretts

Die Funktion `ExtendedBoard` ist der Konstruktor, welcher das ExtendedBoard automatisch mit allen zusätzlich definierten Attributen initialisiert. 

**Input**:
+ board &rarr; das Spielbrett aus der Bibliothek `Chess.jl`

**Output**:
+ das ExtendedBoard

In [10]:
mutable struct ExtendedBoard
    board::Chess.Board
    score::Int32
    endGame::Bool
    repetions::Dict{UInt64, UInt8}
    repetionRuleDraw::Bool
    zobrist::ZobristHash
    
    function ExtendedBoard(board::Chess.Board)::ExtendedBoard
        endGame = isEndGame(board)
        pieceSquareTables = endGame ? PIECE_SQUARE_TABLES_ENDGAME : PIECE_SQUARE_TABLES_MIDGAME
        score = evaluatePositionScore(board, pieceSquareTables)
        return new(board, score, endGame, Dict{UInt64, UInt8}(), false, ZobristHash(board))
    end
end 

Die Funktion `domove!` setzt entweder den vom Spieler gewählten, oder den von der KI berechneten Zug um. 

**Input**:
+ extboard &rarr; das Spielbrett
+ move &rarr; der gewählte Zug

**Output**:
+ ExtendedUndoInfo

In [11]:
function domove!(extboard::ExtendedBoard, move::Chess.Move)::ExtendedUndoInfo
    previousScore = extboard.score
    previousHash = extboard.zobrist.hash
    pieceSquareTables = extboard.endGame ? PIECE_SQUARE_TABLES_ENDGAME : PIECE_SQUARE_TABLES_MIDGAME
    delta = evaluatePositionScoreDeltaIncremental(extboard, move, pieceSquareTables)
    if Chess.sidetomove(extboard.board) == Chess.WHITE
        extboard.score += delta
    else
        extboard.score -= delta
    end
    undo = Chess.domove!(extboard.board, move)
    toggleEnPassant!(extboard.zobrist, Chess.epsquare(extboard.board))
    toggleColor!(extboard.zobrist)
    repetions = get(extboard.repetions, extboard.zobrist.hash, 0) + 1
    extboard.repetions[extboard.zobrist.hash] = repetions
    extboard.repetionRuleDraw = repetions >= 3
    endGame = isEndGame(extboard.board)
    pieceSquareTables = endGame ? PIECE_SQUARE_TABLES_ENDGAME : PIECE_SQUARE_TABLES_MIDGAME
    if endGame != extboard.endGame
        extboard.endGame = endGame
        extboard.score = evaluatePositionScore(extboard.board, pieceSquareTables)
    end
    return (undo, previousScore, previousHash)
end

domove! (generic function with 1 method)

Damit ein Zug zurück genommen werden kann, braucht die Funktion `undomove!` Informationen, welcher Zug zurück genommen werden soll. Diese Informationen werden in der Konstante `ExtendedUndoInfo` gespeichert. Sie wird wie folgt definiert.

In [12]:
if !@isdefined(ExtendedUndoInfo)
    const ExtendedUndoInfo = Tuple{Chess.UndoInfo, Int32, UInt64}
end

Tuple{Chess.UndoInfo, Int32, UInt64}

Die Funktion `undomove!` ermöglicht es einen Zug zurück zu nehmen.

**Input**:
+ extboard &rarr; das Spielbrett der aktuellen Situation
+ undo &rarr; der Kontext des vorhergegangen Zuges

**Output**:
+ das Spielbrett des vorherigen Zuges

In [13]:
function undomove!(extboard::ExtendedBoard, undo::ExtendedUndoInfo)
    extboard.repetions[extboard.zobrist.hash] -= 1
    Chess.undomove!(extboard.board, undo[1])
    extboard.endGame = isEndGame(extboard.board)
    extboard.score = undo[2]
    extboard.zobrist.hash = undo[3]
    extboard.repetionRuleDraw = false
end

undomove! (generic function with 1 method)